In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
}))


In [56]:
f <- '/oak/stanford/groups/mrivas/projects/PRS/private_output/202009_batch/snpnet.eval.2_refit.tsv'
lst_f <- 'GBE_ID.lst'
cat_f <- '/oak/stanford/groups/mrivas/users/ytanigaw/repos/rivas-lab/ukbb-tools/05_gbe/extras/20200812_GBE_category/GBE_category.20201024.tsv'

# output

summary_f <- '/scratch/groups/mrivas/projects/PRS/GBE_data/traits.tsv'


In [39]:
lst_f %>% fread(head=F) %>% pull() -> lst


In [41]:
cat_f %>% 
fread(
    colClasses = c('#GBE_category'='character', 'GBE_ID'='character'),
    select=c('#GBE_category', 'GBE_ID', 'GBE_short_name')
) %>%
rename('GBE_category'='#GBE_category') %>%
rename('trait_name'='GBE_short_name', 'trait_category'='GBE_category') -> cat_df


In [52]:
f %>%
fread(
    colClasses = c('#GBE_ID'='character'),
    select = c(
        '#GBE_ID', 'split', 'geno', 'covar', 'geno_covar', 'geno_delta', 'n_variables', 'family'
    )
) %>%
rename('GBE_ID'='#GBE_ID') %>%
filter(split == 'test', GBE_ID %in% lst) %>% 
unique() %>% select(-split) %>%
left_join(cat_df, by='GBE_ID') %>%
rename('trait'='GBE_ID') %>%
select(
    trait_category, trait, trait_name, family,
    geno, covar, geno_covar, geno_delta, n_variables
) %>% arrange(-geno_delta) -> df 


In [53]:
length(lst)

[1] 1772

In [55]:
df %>% dim()


[1] 1772    9

In [58]:
df %>%
rename('#trait_category' = 'trait_category') %>%
fwrite(summary_f, sep='\t', na = "NA", quote=F)


In [59]:
summary_f

[1] "/scratch/groups/mrivas/projects/PRS/GBE_data/traits.tsv"